##                                        FETCH REWARDS EXERCISE 

I transformed the receipts.json, brands.json, and users.json files into a structured, tabular format for this script. After converting the json files into tabular format, I discovered some data quality issues that the business stakeholder should be made aware of. Using MySQL, I also looked at the query "top 5 brands by receipts scanned for the most recent month" and had the script below created as a guide.

### 1) Review Existing Unstructured Data and Diagram a New Structured Relational Data Model 

The goal is to cleanly transform the unstructured JSON files into structured data that can be queried. To process and clean the data, I'll use pandas. For ER-Diagram, kindly refer to ER_Diagram.pdf

In [1]:
import numpy as np
import pandas as pd
import json
import gzip
import shutil
import os
import pandas as pd
from ast import literal_eval
import matplotlib.pyplot as plt
from datetime import datetime


#### Receipts  Data

In [2]:
# To Extract the Ziped JSON file
# for i in os.listdir():
#     if 'json' in i:
#         with gzip.open(i, 'rb') as f_in:
#             with open(i.replace('.gz',''), 'wb') as f_out:
#                 shutil.copyfileobj(f_in, f_out)

In [3]:
receipts = pd.read_json('receipts.json',lines=True)
brands = pd.read_json('brands.json',lines=True)
users = pd.read_json('users.json',lines=True)

In [4]:
receipts.shape


(1119, 15)

In [5]:
receipts.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1119 entries, 0 to 1118
Data columns (total 15 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   _id                      1119 non-null   object 
 1   bonusPointsEarned        544 non-null    float64
 2   bonusPointsEarnedReason  544 non-null    object 
 3   createDate               1119 non-null   object 
 4   dateScanned              1119 non-null   object 
 5   finishedDate             568 non-null    object 
 6   modifyDate               1119 non-null   object 
 7   pointsAwardedDate        537 non-null    object 
 8   pointsEarned             609 non-null    float64
 9   purchaseDate             671 non-null    object 
 10  purchasedItemCount       635 non-null    float64
 11  rewardsReceiptItemList   679 non-null    object 
 12  rewardsReceiptStatus     1119 non-null   object 
 13  totalSpent               684 non-null    float64
 14  userId                  

The next step is to convert the createDate, dateScanned, finishedDate, modifyDate, pointsAwardedDate, purchaseDate columns to date_time data type. 

In [6]:
def date_convert(x):
    try:
        return(datetime.utcfromtimestamp(int(x['$date'])/1000).strftime('%Y-%m-%d %H:%M:%S'))
    except TypeError:
        return None

receipts['_id'] = receipts['_id'].apply(lambda x: x['$oid'])
receipts['createDate'] = receipts['createDate'].apply(lambda x: date_convert(x))
receipts['dateScanned'] = receipts['dateScanned'].apply(lambda x: date_convert(x))
receipts['finishedDate'] = receipts['finishedDate'].apply(lambda x: date_convert(x))
receipts['modifyDate'] = receipts['modifyDate'].apply(lambda x: date_convert(x))
receipts['pointsAwardedDate'] = receipts['pointsAwardedDate'].apply(lambda x: date_convert(x))
receipts['purchaseDate'] = receipts['purchaseDate'].apply(lambda x: date_convert(x))

In the next step we can seperate the 'rewardsReceiptItemList' field into multiple rows using explode() function as it contains lists of receipts.Using literal_eval, we can evaluate items as dictionary and extract the columns from it using the pandas json normalize method. After that, we rejoin this information to the data frame containing the receipts.

In [7]:
receipts = receipts.explode('rewardsReceiptItemList')
receipts.reset_index(inplace=True)

receipts = receipts.fillna({'rewardsReceiptItemList':'{}'})
receipts['rewardsReceiptItemList'] = receipts['rewardsReceiptItemList'].apply(lambda x:str(x))
receipts['rewardsReceiptItemList'] = receipts['rewardsReceiptItemList'].apply(literal_eval)

receipts_norm = pd.json_normalize(receipts['rewardsReceiptItemList'],errors='ignore',record_prefix='rewardsReceiptItemList')\
.add_prefix('rewards_')

In [8]:
finalReceipts = pd.merge(receipts, receipts_norm, left_index=True, right_index=True, how='outer')
finalReceipts

,index,_id,bonusPointsEarned,bonusPointsEarnedReason,createDate,dateScanned,finishedDate,modifyDate,pointsAwardedDate,pointsEarned,...,rewards_itemNumber,rewards_originalMetaBriteQuantityPurchased,rewards_pointsEarned,rewards_targetPrice,rewards_competitiveProduct,rewards_originalFinalPrice,rewards_originalMetaBriteItemPrice,rewards_deleted,rewards_priceAfterCoupon,rewards_metabriteCampaignId
0,0,5ff1e1eb0a720f0523000575,500.0,"Receipt number 2 completed, bonus point schedu...",2021-01-03 15:25:31,2021-01-03 15:25:31,2021-01-03 15:25:31,2021-01-03 15:25:36,2021-01-03 15:25:31,500.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,5ff1e1bb0a720f052300056b,150.0,"Receipt number 5 completed, bonus point schedu...",2021-01-03 15:24:43,2021-01-03 15:24:43,2021-01-03 15:24:43,2021-01-03 15:24:48,2021-01-03 15:24:43,150.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1,5ff1e1bb0a720f052300056b,150.0,"Receipt number 5 completed, bonus point schedu...",2021-01-03 15:24:43,2021-01-03 15:24:43,2021-01-03 15:24:43,2021-01-03 15:24:48,2021-01-03 15:24:43,150.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2,5ff1e1f10a720f052300057a,5.0,All-receipts receipt bonus,2021-01-03 15:25:37,2021-01-03 15:25:37,None,2021-01-03 15:25:42,None,5.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,3,5ff1e1ee0a7214ada100056f,5.0,All-receipts receipt bonus,2021-01-03 15:25:34,2021-01-03 15:25:34,2021-01-03 15:25:34,2021-01-03 15:25:39,2021-01-03 15:25:34,5.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7376,1115,603d0b710a720fde1000042a,NaN,NaN,2021-03-01 15:42:41,2021-03-01 15:42:41,None,2021-03-01 15:42:41,None,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7377,1116,603cf5290a720fde10000413,NaN,NaN,2021-03-01 14:07:37,2021-03-01 14:07:37,None,2021-03-01 14:07:37,None,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7378,1117,603ce7100a7217c72c000405,25.0,COMPLETE_NONPARTNER_RECEIPT,2021-03-01 13:07:28,2021-03-01 13:07:28,None,2021-03-01 13:07:29,None,25.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,22.97,NaN
7379,1117,603ce7100a7217c72c000405,25.0,COMPLETE_NONPARTNER_RECEIPT,2021-03-01 13:07:28,2021-03-01 13:07:28,None,2021-03-01 13:07:29,None,25.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11.99,NaN


#### Users Data

In [9]:
# extract the id from the dictionary
users['_id'] = users['_id'].apply(lambda x: x['$oid'])
users['createdDate'] = users['createdDate'].apply(lambda x: date_convert(x))
users['lastLogin'] = users['lastLogin'].apply(lambda x: date_convert(x))
     

In [10]:
users

,_id,active,createdDate,lastLogin,role,signUpSource,state
0,5ff1e194b6a9d73a3a9f1052,True,2021-01-03 15:24:04,2021-01-03 15:25:37,consumer,Email,WI
1,5ff1e194b6a9d73a3a9f1052,True,2021-01-03 15:24:04,2021-01-03 15:25:37,consumer,Email,WI
2,5ff1e194b6a9d73a3a9f1052,True,2021-01-03 15:24:04,2021-01-03 15:25:37,consumer,Email,WI
3,5ff1e1eacfcf6c399c274ae6,True,2021-01-03 15:25:30,2021-01-03 15:25:30,consumer,Email,WI
4,5ff1e194b6a9d73a3a9f1052,True,2021-01-03 15:24:04,2021-01-03 15:25:37,consumer,Email,WI
...,...,...,...,...,...,...,...
490,54943462e4b07e684157a532,True,2014-12-19 14:21:22,2021-03-05 16:52:23,fetch-staff,NaN,NaN
491,54943462e4b07e684157a532,True,2014-12-19 14:21:22,2021-03-05 16:52:23,fetch-staff,NaN,NaN
492,54943462e4b07e684157a532,True,2014-12-19 14:21:22,2021-03-05 16:52:23,fetch-staff,NaN,NaN
493,54943462e4b07e684157a532,True,2014-12-19 14:21:22,2021-03-05 16:52:23,fetch-staff,NaN,NaN


#### Brands Data 

In [11]:
brands.head()

,_id,barcode,category,categoryCode,cpg,name,topBrand,brandCode
0,{'$oid': '601ac115be37ce2ead437551'},511111019862,Baking,BAKING,"{'$id': {'$oid': '601ac114be37ce2ead437550'}, ...",test brand @1612366101024,0.0,NaN
1,{'$oid': '601c5460be37ce2ead43755f'},511111519928,Beverages,BEVERAGES,"{'$id': {'$oid': '5332f5fbe4b03c9a25efd0ba'}, ...",Starbucks,0.0,STARBUCKS
2,{'$oid': '601ac142be37ce2ead43755d'},511111819905,Baking,BAKING,"{'$id': {'$oid': '601ac142be37ce2ead437559'}, ...",test brand @1612366146176,0.0,TEST BRANDCODE @1612366146176
3,{'$oid': '601ac142be37ce2ead43755a'},511111519874,Baking,BAKING,"{'$id': {'$oid': '601ac142be37ce2ead437559'}, ...",test brand @1612366146051,0.0,TEST BRANDCODE @1612366146051
4,{'$oid': '601ac142be37ce2ead43755e'},511111319917,Candy & Sweets,CANDY_AND_SWEETS,"{'$id': {'$oid': '5332fa12e4b03c9a25efd1e7'}, ...",test brand @1612366146827,0.0,TEST BRANDCODE @1612366146827


In [20]:
brands['_id'] = brands['_id'].apply(lambda x: x['$oid'])

In [27]:
brands_norm = pd.json_normalize(brands['cpg'])
brands_norm = brands_norm.add_prefix('cpg_')

FinalBrands = pd.merge(brands, brands_norm, left_index=True, right_index=True, how='outer')

In [29]:
FinalBrands.drop('cpg',axis = 1, inplace = True)

In [30]:
FinalBrands

,_id,barcode,category,categoryCode,name,topBrand,brandCode,cpg_$ref,cpg_$id.$oid
0,601ac115be37ce2ead437551,511111019862,Baking,BAKING,test brand @1612366101024,0.0,NaN,Cogs,601ac114be37ce2ead437550
1,601c5460be37ce2ead43755f,511111519928,Beverages,BEVERAGES,Starbucks,0.0,STARBUCKS,Cogs,5332f5fbe4b03c9a25efd0ba
2,601ac142be37ce2ead43755d,511111819905,Baking,BAKING,test brand @1612366146176,0.0,TEST BRANDCODE @1612366146176,Cogs,601ac142be37ce2ead437559
3,601ac142be37ce2ead43755a,511111519874,Baking,BAKING,test brand @1612366146051,0.0,TEST BRANDCODE @1612366146051,Cogs,601ac142be37ce2ead437559
4,601ac142be37ce2ead43755e,511111319917,Candy & Sweets,CANDY_AND_SWEETS,test brand @1612366146827,0.0,TEST BRANDCODE @1612366146827,Cogs,5332fa12e4b03c9a25efd1e7
...,...,...,...,...,...,...,...,...,...
1162,5f77274dbe37ce6b592e90c0,511111116752,Baking,BAKING,test brand @1601644365844,NaN,NaN,Cogs,5f77274dbe37ce6b592e90bf
1163,5dc1fca91dda2c0ad7da64ae,511111706328,Breakfast & Cereal,NaN,Dippin Dots® Cereal,NaN,DIPPIN DOTS CEREAL,Cogs,53e10d6368abd3c7065097cc
1164,5f494c6e04db711dd8fe87e7,511111416173,Candy & Sweets,CANDY_AND_SWEETS,test brand @1598639215217,NaN,TEST BRANDCODE @1598639215217,Cogs,5332fa12e4b03c9a25efd1e7
1165,5a021611e4b00efe02b02a57,511111400608,Grocery,NaN,LIPTON TEA Leaves,0.0,LIPTON TEA Leaves,Cogs,5332f5f6e4b03c9a25efd0b4


### 2 ) Write a query that directly answers a predetermined question from a business stakeholder 

In [31]:
!pip install pandasql
from pandasql import sqldf
pysqldf = lambda q: sqldf(q, globals())

SQL query against my new structured relational data model that answers few questions

Que 1. When considering average spend from receipts with 'rewardsReceiptStatus’ of ‘Accepted’ or ‘Rejected’, which is greater?

Ans) Assuming ' FINISHED' as 'Accepted', 'Accepted' is greater when considering average spend from receipts with 'rewardsReceiptStatus’

In [34]:
Receipts_copy = finalReceipts.copy()
Receipts_copy = Receipts_copy.applymap(str)
q = """SELECT r.rewardsReceiptStatus as Rewards_Receipt_Status, avg(r.totalSpent) as Average_Total_Spent FROM Receipts_copy r GROUP BY r.rewardsReceiptStatus"""

data = pysqldf(q)
data

,Rewards_Receipt_Status,Average_Total_Spent
0,FINISHED,1244.372934
1,FLAGGED,2635.570247
2,PENDING,27.471800
3,REJECTED,19.544970
4,SUBMITTED,0.000000


Que 2. When considering total number of items purchased from receipts with 'rewardsReceiptStatus’ of ‘Accepted’ or ‘Rejected’, which is greater?

Ans) When considering total number of items purchased from receipts with 'rewardsReceiptStatus’ of ‘Accepted’ or ‘Rejected’, 'Accepted' is greater

In [36]:
q = """SELECT r.rewardsReceiptStatus, count(r.purchasedItemCount) as Total_Count FROM Receipts_copy r GROUP BY r.rewardsReceiptStatus;"""

data1 = pysqldf(q)
data1

,rewardsReceiptStatus,Total_Count
0,FINISHED,5920
1,FLAGGED,810
2,PENDING,50
3,REJECTED,167
4,SUBMITTED,434


### 3 ) Evaluate Data Quality Issues in the Data Provided

I skimmed through the null values for each table and the numerical distribution in this section.

It turns out that several columns in these data frames have a high percentage of missing values, which means that a lot of data is missing. In some situations, this might be expected; for instance, a user might choose not to flag an item. As a result, the columns relating to user flags continue to have significant NA values. It might, however, present issues in some circumstances. For instance, the barCode will be the join key when joining the items table to the brands table. As a result, the join of the two data may result in inaccurate information because the barCode column in the items table has quite a few missing entries.Now we contrast the two to see whether we can map the brand codes on all of the receipts to the brands table.

In [89]:
len(set(finalReceipts.dropna(subset=['rewards_brandCode'])['rewards_brandCode']))

227

In [92]:
len(set(FinalBrands.dropna(subset=['brandCode'])['brandCode']))

897

In [94]:
receipts_dropped = finalReceipts.dropna(subset=['rewards_brandCode'])
receipts_dropped['rewards_brandCode'] = receipts_dropped['rewards_brandCode'].apply(lambda x:str(x))

/var/folders/q4/2s68lhzs5nqdpj98_gdtpxtc0000gn/T/ipykernel_17741/2095294277.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  receipts_dropped['rewards_brandCode'] = receipts_dropped['rewards_brandCode'].apply(lambda x:str(x))


In [95]:
FinalBrands['brandCode'] = FinalBrands['brandCode'].apply(lambda x: str(x))
receipts_with_brands = pd.merge(finalReceipts,FinalBrands,left_on='rewards_brandCode',right_on='brandCode')

In [105]:
set(receipts_dropped[~(receipts_dropped['rewards_brandCode'].isin(brands['brandCode']))]['rewards_brandCode'])
     

{'7UP',
 'ADVIL',
 'AMERICAN BEAUTY',
 'ARROWHEAD',
 'AZTECA',
 'BANZA',
 'BEAR CREEK COUNTRY KITCHENS',
 'BEN AND JERRYS',
 'BETTY CROCKER',
 'BIC',
 'BIGELOW',
 'BLUE DIAMOND',
 "BOAR'S HEAD",
 'BORDEN',
 'BOTA BOX',
 'BRAND',
 "BRASWELL'S",
 'BUNNY',
 "BUSH'S BEST",
 'C&H',
 'CADBURY',
 'CAL-ORGANIC FARMS',
 'CALIFIA FARMS',
 "CAMPBELL'S",
 'CARAMELLO',
 'CHEERIOS',
 'CHEESE',
 'CHEEZ-IT',
 'CHEX',
 'CHICKEN OF THE SEA',
 'CHIQUITA',
 'CINNAMON TOAST CRUNCH',
 'COKE',
 'COLEMAN NATURAL',
 "CONNIE'S PIZZA",
 'CREST 3D WHITE',
 'CRISPIX',
 'DANNON',
 'DARE',
 'DELI',
 'DIET COKE',
 'DIGIORNO',
 'DOLE',
 'DR PEPPER',
 'EDWARDS',
 "EGGLAND'S BEST",
 'EGGO',
 'EL MONTEREY',
 'ENERGIZER MAX',
 'ESSENTIAL EVERYDAY',
 'FAGE',
 "FAMOUS DAVE'S",
 "FLORIDA'S NATURAL",
 'FOLGERS',
 'FORTUNE YAKISOBA',
 'FRANZ',
 "FRENCH'S",
 'FRESH EXPRESS',
 'FRESH STEP',
 'FRONTERA',
 'GALLO FAMILY VINEYARDS',
 'GENERAL MILLS',
 'GERBER',
 'GERM-X',
 'GREEN GIANT',
 'GRIMMWAY FARMS',
 'HANOVER',
 'HARVEST SNA

All of the receipts with brand codes are shown above, however the codes are not present in the brands table. This would be a problem since when we query between the two tables, a lot of data will be missing.

In [108]:
finalReceipts.isnull().sum()

index                                            0
_id                                              0
bonusPointsEarned                             1401
bonusPointsEarnedReason                       1401
createDate                                       0
dateScanned                                      0
finishedDate                                  1411
modifyDate                                       0
pointsAwardedDate                             1301
pointsEarned                                  1128
purchaseDate                                   458
purchasedItemCount                             484
rewardsReceiptItemList                           0
rewardsReceiptStatus                             0
totalSpent                                     435
userId                                           0
rewards_barcode                               4291
rewards_description                            821
rewards_finalPrice                             614
rewards_itemPrice              